# Exercise 5.2

First of all, we import usefull libraries.

In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import cvxopt

Now, we define a function to choose set of training or testing data for data_set and labels, as well as another one to obtain length of labels array.

In [2]:
def choose_set_for_label(data_set, labels, i):
    train_data_set = data_set[positions_train]
    train_labels = labels[positions_train]
    
    test_data_set = data_set[positions_test]
    test_labels = labels[positions_test]
    
    train_data_set = train_data_set[train_labels!=i]
    train_labels = train_labels[train_labels!=i]
    
    test_data_set = test_data_set[test_labels!=i]
    test_labels = test_labels[test_labels!=i]
    
    test_labels[test_labels == np.min(test_labels)] = -1
    test_labels[test_labels == np.max(test_labels)] = 1
    
    train_labels[train_labels == np.min(train_labels)] = -1
    train_labels[train_labels == np.max(train_labels)] = 1
    return train_data_set, test_data_set, train_labels, test_labels

def get_labels_count(data_set):
    labels_count = len(data_set)
    return labels_count

Use the code that we have implemented earlier:

In [3]:
def train(train_data_set, train_labels, kernel_type='linear', C=10, threshold=1e-5):
    kernel = build_kernel(train_data_set, kernel_type=kernel_type)

    P = train_labels * train_labels.transpose() * kernel
    q = -np.ones((objects_count, 1))
    G = np.concatenate((np.eye(objects_count), -np.eye(objects_count)))
    h = np.concatenate((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))

    A = train_labels.reshape(1, objects_count)
    A = A.astype(float)
    b = 0.0

    sol = cvxopt.solvers.qp(cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h), cvxopt.matrix(A), cvxopt.matrix(b))

    lambdas = np.array(sol['x'])

    support_vectors_id = np.where(lambdas > threshold)[0]
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    b /= len(lambdas)

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number

def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    return kernel

def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    kernel = np.dot(test_data_set, support_vectors.T)
    sigma = 1.0
    #K = np.dot(test_data_set, support_vectors.T)
    #kernel = build_kernel(train_data_set, kernel_type='rbf')
    c = (1. / sigma * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
    c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
    #aa = np.dot((np.diag(K)*np.ones((1,len(test_data_set)))).T[support_vectors_id], np.ones((1, np.shape(K)[0]))).T
    sv = (np.diag(np.dot(train_data_set, train_data_set.T))*np.ones((1,len(train_data_set)))).T[support_vectors_id]
    aa = np.dot(sv,np.ones((1,np.shape(kernel)[0]))).T
    kernel = kernel - 0.5 * c - 0.5 * aa
    kernel = np.exp(kernel / (2. * sigma ** 2))

    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    return np.sign(y)

We load the data and apply the functions we defined before. We run three times the code, so we obtain the predicted results from the three pairs. Then, we apply a voting method to decide the correct value from the three sets and, finally, obtain the accuracy.

Now, we solved problem we had and we could generalizate the method as we wanted to.

In [4]:
iris = load_iris()
data_set = iris.data
labels = iris.target

predictions_0_1 = []; predictions_0_2 = []; predictions_1_2 = []

alpha = 0.8

positions_train = np.random.choice(range(len(labels)), int(len(labels) * alpha), replace = False)
positions_test = np.delete(range(len(labels)), positions_train)
test_data_set = data_set[positions_test]
test_labels = labels[positions_test]

train_data_set_0_1, test_data_set_0_1, train_labels_0_1, test_labels_0_1 = choose_set_for_label(data_set, labels, 2)
train_data_set_0_2, test_data_set_0_2, train_labels_0_2, test_labels_0_2 = choose_set_for_label(data_set, labels, 1)
train_data_set_1_2, test_data_set_1_2, train_labels_1_2, test_labels_1_2 = choose_set_for_label(data_set, labels, 0)

# Labels with values 0 and 1

objects_count = get_labels_count(train_labels_0_1)
lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(train_data_set_0_1, train_labels_0_1, kernel_type='rbf')
predicted = classify_rbf(test_data_set, train_data_set_0_1, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
predicted = list(predicted.astype(int))
predictions_0_1.append(predicted)

nc, nf, o = np.shape(predictions_0_1)
predictions_0_1 = np.reshape(predictions_0_1, (nf, nc))

predictions_0_1[predictions_0_1 == -1] = 0

# Labels with values 0 and 2

objects_count = get_labels_count(train_labels_0_2)
lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(train_data_set_0_2, train_labels_0_2, kernel_type='rbf')
predicted = classify_rbf(test_data_set, train_data_set_0_2, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
predicted = list(predicted.astype(int))
predictions_0_2.append(predicted)

nc, nf, o = np.shape(predictions_0_2)
predictions_0_2 = np.reshape(predictions_0_2, (nf, nc))

predictions_0_2[predictions_0_2 == 1] = 2
predictions_0_2[predictions_0_2 == -1] = 0

# Labels with values 1 and 2

objects_count = get_labels_count(train_labels_1_2)
lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(train_data_set_1_2, train_labels_1_2, kernel_type='rbf')
predicted = classify_rbf(test_data_set, train_data_set_1_2, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
predicted = list(predicted.astype(int))
predictions_1_2.append(predicted)

nc, nf, o = np.shape(predictions_1_2)
predictions_1_2 = np.reshape(predictions_1_2, (nf, nc))

predictions_1_2[predictions_1_2 == 1] = 2
predictions_1_2[predictions_1_2 == -1] = 1

# We append all peditions

preditions = np.append(np.append(predictions_0_1, predictions_0_2), predictions_1_2)
preditions = np.reshape(preditions, (nf, 3))

# We apply a voting method in order to decide 'true' labels

final_predition = np.zeros(nf)

for i in range(nf):
    aux = preditions[i]
    final_predition[i] = np.argmax(np.bincount(aux))

#print('Obtained predition is:', final_predition)
print('The accuracy is:', accuracy_score(final_predition, test_labels))

     pcost       dcost       gap    pres   dres
 0:  9.5267e+01 -1.1859e+03  2e+03  2e-01  2e-15
 1:  5.6830e+01 -1.1149e+02  2e+02  4e-03  2e-15
 2:  6.5878e+00 -1.5743e+01  2e+01  6e-16  2e-15
 3: -5.5105e-01 -3.6109e+00  3e+00  2e-16  7e-16
 4: -1.1484e+00 -1.7499e+00  6e-01  2e-16  3e-16
 5: -1.2754e+00 -1.5344e+00  3e-01  2e-16  2e-16
 6: -1.3694e+00 -1.4749e+00  1e-01  2e-16  2e-16
 7: -1.4071e+00 -1.4310e+00  2e-02  2e-16  2e-16
 8: -1.4151e+00 -1.4175e+00  2e-03  2e-16  2e-16
 9: -1.4160e+00 -1.4161e+00  7e-05  2e-16  2e-16
10: -1.4161e+00 -1.4161e+00  2e-06  2e-16  2e-16
11: -1.4161e+00 -1.4161e+00  1e-07  6e-16  2e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  1.0813e+02 -1.2374e+03  2e+03  2e-01  2e-15
 1:  6.0615e+01 -1.2296e+02  2e+02  5e-03  2e-15
 2:  6.8820e+00 -1.7307e+01  2e+01  7e-16  2e-15
 3: -7.6822e-01 -4.0557e+00  3e+00  2e-16  6e-16
 4: -1.4045e+00 -2.2164e+00  8e-01  2e-16  2e-16
 5: -1.5991e+00 -1.9084e+00  3e-01  2e-16  2e-1